<a href="https://colab.research.google.com/github/Tariquzzaman-faisal/VITD/blob/main/mbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Mon Aug 28 11:03:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Project Setup

In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.4 MB/s eta 0:00:00


In [4]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

In [5]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained mBERT model and tokenizer

In [11]:
model_name = 'bert-base-multilingual-cased'
num_classes = 3  # Update with the number of classes in your task
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Loading dataset

In [12]:
# Load CSV data
dir = '/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/dataset/Tariq/split/'
train_data = pd.read_csv(f'{dir}train.csv')
test_data = pd.read_csv(f'{dir}test.csv')
val_data = pd.read_csv(f'{dir}validation.csv')

# Model setup

In [13]:
def preprocess_data(data, max_length):
    encodings = tokenizer(data['sentence1'].tolist(), truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
    labels = torch.tensor(data['label'].tolist()).to(device)
    dataset = TensorDataset(encodings.input_ids.to(device), encodings.attention_mask.to(device), labels)
    return dataset

In [14]:
max_length = 256  # Set your desired max sequence length
train_dataset = preprocess_data(train_data, max_length)
val_dataset = preprocess_data(val_data, max_length)
test_dataset = preprocess_data(test_data, max_length)

In [15]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [16]:
# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training (skip this for saved model)

In [ ]:
# Move the model to the same device as device
model = model.to(device)

In [ ]:
# Define early stopping parameters
early_stopping_patience = 3
best_val_loss = float('inf')
epochs_since_last_improvement = 0

# Training loop
num_epochs = 20  # Increased the number of epochs
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()
            predicted_labels = torch.argmax(outputs.logits, dim=1)
            correct += (predicted_labels == labels).sum().item()
            total += labels.size(0)

    val_accuracy = correct / total
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Avg. Validation Loss: {avg_val_loss:.4f}")

    # Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_since_last_improvement = 0
    else:
        epochs_since_last_improvement += 1
        if epochs_since_last_improvement >= early_stopping_patience:
            print("Early stopping triggered. Stopping training.")
            break


Epoch [1/20]
Validation Accuracy: 0.7128
Avg. Validation Loss: 0.7064
Epoch [2/20]
Validation Accuracy: 0.7353
Avg. Validation Loss: 0.6349
Epoch [3/20]
Validation Accuracy: 0.7090
Avg. Validation Loss: 0.7366
Epoch [4/20]
Validation Accuracy: 0.7301
Avg. Validation Loss: 0.8490
Epoch [5/20]
Validation Accuracy: 0.7444
Avg. Validation Loss: 0.7907
Early stopping triggered. Stopping training.


# Saving tuned model

In [ ]:
from transformers import AutoModel, AutoConfig

# Define the path to the saved model
model_path = "/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/models/mbert/mbert_trained_model.pth"

# Save the model
torch.save(model.state_dict(), model_path)

In [ ]:
# Create an instance of the model
model_name = 'bert-base-multilingual-cased'
num_classes = 3  # Update with the number of classes in your task
model_path = "/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/models/mbert/mbert_trained_model.pth"

model2 = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

# Load the saved parameters into the model
model2.load_state_dict(torch.load(model_path))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

# Report on test set

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
# Evaluate on the test set
model2.to(device)  # Move the model to the same device as tensors
model2.eval()
test_labels_list = []
predicted_labels_list = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        outputs = model2(input_ids, attention_mask=attention_mask)
        predicted_labels = torch.argmax(outputs.logits, dim=1)

        test_labels_list.extend(labels.cpu().numpy())
        predicted_labels_list.extend(predicted_labels.cpu().numpy())

test_labels_np = np.array(test_labels_list)
predicted_labels_np = np.array(predicted_labels_list)

In [ ]:
# Generate classification report
from sklearn.metrics import classification_report
target_names = ["0", "1", "2"]  # Replace with your class names
report = classification_report(test_labels_np, predicted_labels_np, target_names=target_names)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       1.00      0.64      0.78      2016
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0

    accuracy                           0.64      2016
   macro avg       0.33      0.21      0.26      2016
weighted avg       1.00      0.64      0.78      2016



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Create a DataFrame with index and prediction columns
result_df = pd.DataFrame({
    'index': range(len(test_labels_np)),
    'prediction': predicted_labels_np
})

# Save the DataFrame to a CSV file
result_df.to_csv('/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/models/mbert/predictions.csv', index=False)
print("Predictions saved to 'predictions.csv'")

Predictions saved to 'predictions.csv'


# Probability ensemble

In [17]:
import torch
import numpy as np
import pandas as pd

model_name = 'bert-base-multilingual-cased'
num_classes = 3  # Update with the number of classes in your task
model_path = "/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/models/mbert/mbert_trained_model.pth"
model2 = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

# Load the saved parameters into the model
model2.load_state_dict(torch.load(model_path))
model2.to(device)  # Move the model to the same device as tensors
model2.eval()

class_probs_list = []  # List to store predicted class probabilities

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, _ = batch  # No need for labels in this case
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        outputs = model2(input_ids, attention_mask=attention_mask)
        predicted_probabilities = torch.softmax(outputs.logits, dim=1)

        class_probs_list.extend(predicted_probabilities.cpu().numpy())

class_probs_np = np.array(class_probs_list)

# Now you have an array of class probabilities for each example
# class_probs_np.shape will be (num_examples, num_classes)

# Create a DataFrame with index and predicted class probabilities
result_df = pd.DataFrame(class_probs_np, columns=[f'Class_{i}' for i in range(num_classes)])
result_df['index'] = range(len(class_probs_np))

# Reorder the columns so 'index' comes first
result_df = result_df[['index'] + [f'Class_{i}' for i in range(num_classes)]]

# Save the DataFrame to a CSV file
result_df.to_csv('/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/models/mbert/probability_ensemble.csv', index=False)
print("Probabilities saved to probability_ensemble.csv")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Probabilities saved to probability_ensemble.csv
